In [1]:
# !git clone --recursive https://github.com/Microsoft/LightGBM
# ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
from google.colab import drive
drive.mount('/content/drive')

!pip uninstall lightgbm
!pip install lightgbm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from lightgbm import LGBMRegressor

!pip install optuna
import optuna
from functools import partial

import warnings
warnings.filterwarnings('ignore')

import warnings
warnings.filterwarnings(action='ignore')

from tqdm.auto import tqdm


from sklearn.preprocessing import StandardScaler, PowerTransformer,QuantileTransformer ,LabelEncoder, OneHotEncoder, MinMaxScaler, OrdinalEncoder, RobustScaler

Mounted at /content/drive
Uninstalling lightgbm-2.2.3:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/lightgbm-2.2.3.dist-info/*
    /usr/local/lib/python3.6/dist-packages/lightgbm/*
Proceed (y/n)? y
  Successfully uninstalled lightgbm-2.2.3
     |████████████████████████████████| 1.8MB 6.9MB/s 
     |████████████████████████████████| 296kB 5.3MB/s 
     |████████████████████████████████| 81kB 6.4MB/s 
     |████████████████████████████████| 1.2MB 16.4MB/s 
     |████████████████████████████████| 143kB 49.3MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 112kB 37.6MB/s 
     |████████████████████████████████| 481kB 28.8MB/s 
  Created wheel for alembic: filename=alembic-1.5.5-py2.py3-none-any.whl size=156597 sha256=8e6951f16a6f8ac805031aa550e7ed20f848d6d8593bce7d8674ff7a4633583e
  Stored in directory: /root/.cache/pip/wheels/74/3f/61/7de6e3cef766d2680a5d81b1a388286e640f6a681eb589d643
  Created wheel for Mako: filename=Mako-

In [2]:
path = '/content/drive/Shareddrives/dacon/Playground/'
train = pd.read_csv(path + 'train.csv', index_col=['id'])
test = pd.read_csv(path + 'test.csv', index_col=['id'])
submission = pd.read_csv(path + 'sample_submission.csv', index_col=['id'])

In [3]:
path2 = '/content/drive/Shareddrives/dacon/Playground/optimize/'
model3_train = pd.read_csv(path2 + 'trains_optuna_3model.csv', index_col=['Unnamed: 0'])
model3_predict = pd.read_csv(path2 + 'predictions_optuna_3model.csv', index_col=['Unnamed: 0'])

In [4]:
def create_stratified_folds_for_regression(data_df, n_splits=5):
    """
    @param data_df: training data to split in Stratified K Folds for a continous target value
    @param n_splits: number of splits
    @return: the training data with a column with kfold id
    """
    data_df['kfold'] = -1
    # randomize the data
    data_df = data_df.sample(frac=1, random_state=42).reset_index(drop=True)
    # calculate the optimal number of bins based on log2(data_df.shape[0])
    num_bins = np.int(np.floor(1 + np.log2(len(data_df))))
    # bins value will be the equivalent of class value of target feature used by StratifiedKFold to 
    # distribute evenly the classed over each fold
    data_df.loc[:, "bins"] = pd.cut(pd.to_numeric(data_df['target'], downcast="signed"), bins=num_bins, labels=False)
    kf = StratifiedKFold(n_splits=n_splits, random_state=42)
    
    # set the fold id as a new column in the train data
    for f, (t_, v_) in enumerate(kf.split(X=data_df, y=data_df.bins.values)):
        data_df.loc[v_, 'kfold'] = f
    
    # drop the bins column (no longer needed)
    data_df = data_df.drop("bins", axis=1)
    
    return data_df

In [5]:
def cv_evaluation_regression(df, model):
    n_fold = 5
    oof = np.zeros((len(df),))
    for fold in tqdm(range(n_fold)):
        val_ind = df[df.kfold == fold].index
        train = df[df.kfold != fold].reset_index(drop=True)
        valid = df[df.kfold == fold].reset_index(drop=True)

        features = [feature for feature in df.columns if feature not in ['id', 'target', 'kfold']]

        X_train = train[features]
        y_train = train['target']
        X_valid = valid[features]
        y_valid = valid['target']

        model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 500, verbose=False)
        preds = model.predict(valid[features])
        mse = mean_squared_error(y_valid, preds)
        
        oof[val_ind] = preds
    
    mse_oof = mean_squared_error(oof, df['target'])
    
    print(f'LGBM Overall RMSE:{np.sqrt(mse_oof)}')
    return np.sqrt(mse_oof)

In [6]:
train_data = train.iloc[:,:-1]
train_target = train.iloc[:, -1]
test_data = test.copy()

In [7]:
def dropper(dataset):
  df = dataset.copy()
  df['cat3'] = df['cat3'].apply(lambda x: x if x in ['A', 'C'] else 'B')
  df['cat5'] = df['cat5'].apply(lambda x: x if x == 'C' else 'B')
  df['cat6'] = df['cat6'].apply(lambda x: x if x in ['A', 'B', 'C'] else 'D')
  df['cat7'] = df['cat7'].apply(lambda x: 'G' if x in ['G', 'E'] else x)
  df['cat7'] = df['cat7'].apply(lambda x: 'D' if x in ['B', 'D'] else x)
  df['cat8'] = df['cat8'].apply(lambda x: 'A' if x in ['A', 'D'] else x)
  df['cat9'] = df['cat9'].apply(lambda x: 'B' if x in ['O', 'B', 'K'] else x)
  df['cat9'] = df['cat9'].apply(lambda x: 'H' if x in ['N', 'H', 'M'] else x)
  return df

In [8]:
train_temp = train_data.copy()
test_temp = test_data.copy()

cat_features = [feature for feature in train_temp.columns if 'cat' in feature]
cont_features = [feature for feature in train_temp.columns if 'cont' in feature]
scaler  = QuantileTransformer(random_state=42, n_quantiles=1500)

train_data[cont_features] = scaler.fit_transform(train_data[cont_features])
test_data[cont_features] = scaler.transform(test_data[cont_features])

train_data[cat_features] = dropper(train_temp[cat_features])
test_data[cat_features] = dropper(test_data[cat_features])

train_data = pd.concat([train_data, train_target], axis = 1)

In [9]:
train_data

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,,,,,
1,A,B,A,A,B,B,A,G,C,I,0.940916,0.743303,0.046189,0.262185,0.346990,0.900497,0.531811,0.895649,0.932840,0.844205,0.861846,0.678347,0.944570,0.721503,6.994023
2,B,A,A,A,B,B,A,G,A,F,0.310632,0.015017,0.343941,0.973954,0.376355,0.495759,0.274397,0.253887,0.782310,0.590147,0.669928,0.800107,0.220230,0.922358,8.071256
3,A,A,A,C,B,B,A,D,C,H,0.830057,0.895601,0.493890,0.936372,0.409145,0.940410,0.362533,0.944315,0.888857,0.926020,0.142713,0.655557,0.930102,0.977543,5.760456
4,A,A,A,C,B,B,A,G,G,B,0.807019,0.937972,0.082587,0.770248,0.891234,0.962538,0.726620,0.544336,0.891863,0.816634,0.480207,0.664320,0.844488,0.582012,7.806457
6,A,B,A,A,B,B,A,G,C,F,0.470524,0.625662,0.598749,0.447376,0.188841,0.361440,0.837483,0.415459,0.546018,0.473193,0.861935,0.618260,0.317074,0.276864,6.868974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,A,B,A,C,B,B,A,G,E,L,0.059270,0.779462,0.093144,0.586359,0.889268,0.178653,0.064763,0.508377,0.223419,0.203659,0.102613,0.374541,0.006685,0.871086,8.343538
499996,A,B,A,C,B,B,A,G,E,L,0.019205,0.126351,0.797717,0.158230,0.900239,0.026820,0.062324,0.688836,0.114638,0.158871,0.173308,0.264305,0.132952,0.103993,7.851861
499997,A,B,A,C,B,B,A,G,C,H,0.203554,0.549040,0.514314,0.557816,0.413842,0.519039,0.422331,0.599026,0.046766,0.520089,0.102704,0.249794,0.017893,0.372121,7.600558


In [10]:
from sklearn import preprocessing

In [11]:
for cat in cat_features:
  le = preprocessing.LabelEncoder()
  le.fit(train_data[cat])
  train_data[cat] = le.transform(train_data[cat])
  test_data[cat] = le.transform(test_data[cat])

In [12]:
# Outlier 제거
trainset = train_data.copy()
trainset.drop(['target'], axis = 1, inplace = True)

from sklearn.ensemble import IsolationForest
iso_model = IsolationForest(max_samples='auto', random_state=42, n_jobs = -1, contamination = 0.05, n_estimators=500)
iso_model.fit(trainset)

score = iso_model.decision_function(trainset)
train_data['scores'] = score

tcore = iso_model.decision_function(test_data)
test_data['scores'] = tcore

In [13]:
test_data

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,scores
id,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,2,1,0,0,4,3,6,0.181297,0.422660,0.574301,0.012619,0.798934,0.670746,0.109528,0.260123,0.097120,0.057310,0.172093,0.162777,0.626920,0.293701,0.074612
5,0,1,0,2,1,0,0,4,2,10,0.283044,0.541797,0.216367,0.127725,0.072428,0.578952,0.460720,0.599493,0.879462,0.541190,0.554420,0.779753,0.180078,0.762033,0.072275
15,0,1,0,2,1,0,0,4,2,5,0.651992,0.554209,0.940422,0.187473,0.222654,0.726137,0.828580,0.119896,0.776960,0.594316,0.948823,0.820309,0.259189,0.177695,0.083985
16,0,0,1,0,1,0,0,4,3,5,0.313816,0.396663,0.915738,0.153593,0.628850,0.799085,0.041254,0.328692,0.551564,0.650312,0.384003,0.147819,0.578299,0.319104,0.033331
17,0,1,0,0,1,0,0,4,3,8,0.272483,0.864629,0.255125,0.148694,0.871840,0.046333,0.217553,0.724498,0.072970,0.072986,0.824519,0.096540,0.069721,0.875406,0.030364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499987,0,0,0,2,1,0,0,4,5,10,0.937143,0.117951,0.143247,0.098501,0.067358,0.983485,0.041723,0.833123,0.959484,0.977521,0.442583,0.520080,0.922638,0.493020,0.020892
499990,0,0,0,2,1,0,0,4,3,5,0.237333,0.001155,0.529949,0.406330,0.687811,0.093621,0.338246,0.844916,0.047461,0.151081,0.300653,0.383587,0.636118,0.089604,0.084255
499991,0,0,0,2,1,0,0,4,2,1,0.372978,0.817326,0.475959,0.549943,0.680300,0.481162,0.921397,0.424252,0.698674,0.577772,0.450975,0.564149,0.768653,0.513779,0.103288


In [20]:
lgbm_params = {'cat_l2': 24.16256018684644, 
                'cat_smooth': 19, 
                'colsample_bytree': 0.18021264437889178, 
                'learning_rate': 0.012,
                'max_bin': 589, 
                'max_depth': 22, 
                'min_child_samples': 255, 
                'min_child_weight': 17.408465847532234, 
                'n_estimators': 8000, 
                'num_leaves': 65, 
                'reg_alpha': 22.912590049548058, 
                'reg_lambda': 18.645825925028017, 
                'n_jobs': -1, 
                'subsample': 0.990962372303398,
                'random_state':42,
                'metric' : 'rmse'}

In [21]:
X_train = train_data.drop(['target'], axis = 1)
y_train = train.target
X_test = test_data.copy()

In [16]:
X_train = train.drop(['target'], axis = 1)
y_train = train.target
X_test = test.copy()

In [17]:
cat_cols = [feature for feature in train.columns if 'cat' in feature]

def label_encoder(df):
    for feature in cat_cols:
        le = LabelEncoder()
        le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df
X_train = label_encoder(X_train)
X_test = label_encoder(X_test)

In [18]:
from sklearn.model_selection import KFold, GridSearchCV, cross_validate, train_test_split

In [19]:
split = KFold(n_splits=5, random_state=42)

In [ ]:
aa

In [23]:
preds_list_base = []
preds_list_final_iteration = []
preds_list_all = []

for train_idx, val_idx in split.split(X_train):
            X_tr = X_train.iloc[train_idx]
            X_val = X_train.iloc[val_idx]
            y_tr = y_train.iloc[train_idx]
            y_val = y_train.iloc[val_idx]
            
            Model = LGBMRegressor(**lgbm_params).fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                          eval_metric=['rmse'],
                          early_stopping_rounds=250, 
                          categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                          #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
                          verbose=0)
            
            preds_list_base.append(Model.predict(X_test))
            preds_list_all.append(Model.predict(X_test))
            print(f'RMSE for Base model is {np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))}')
            first_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
            params = lgbm_params.copy()
            
            for i in range(1, 8):
                if i >2:    
                    
                    # reducing regularizing params if 
                    
                    params['reg_lambda'] *= 0.9
                    params['reg_alpha'] *= 0.9
                    params['num_leaves'] += 40
                    params['cat_l2'] *= 0.9
                    
                params['learning_rate'] = 0.003
                Model = LGBMRegressor(**params).fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                          eval_metric=['rmse'],
                          early_stopping_rounds=200, 
                          categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                          verbose=0, init_model = Model)
                
                preds_list_all.append(Model.predict(X_test))
                print(f'RMSE for Incremental trial {i} model is {np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))}')
            last_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
            print('',end='\n\n')
            print(f'Improvement of : {first_rmse - last_rmse}')
            print('-' * 100)
            preds_list_final_iteration.append(Model.predict(X_test))

RMSE for Base model is 0.841709132519091
RMSE for Incremental trial 1 model is 0.8417047644260093
RMSE for Incremental trial 2 model is 0.8416937176723688
RMSE for Incremental trial 3 model is 0.8416911133574166
RMSE for Incremental trial 4 model is 0.841690041063156
RMSE for Incremental trial 5 model is 0.8416903661180162
RMSE for Incremental trial 6 model is 0.8416908297801995
RMSE for Incremental trial 7 model is 0.8416917299399458


Improvement of : 1.7402579145175956e-05
----------------------------------------------------------------------------------------------------
RMSE for Base model is 0.8414790845314805
RMSE for Incremental trial 1 model is 0.8414751166684226
RMSE for Incremental trial 2 model is 0.8414722703065934
RMSE for Incremental trial 3 model is 0.841469463723751
RMSE for Incremental trial 4 model is 0.8414654083659455
RMSE for Incremental trial 5 model is 0.8414591795974242
RMSE for Incremental trial 6 model is 0.8414567203449549
RMSE for Incremental trial 7 model 

In [ ]:
preds_list_base = []
preds_list_final_iteration = []
preds_list_all = []

for train_idx, val_idx in split.split(X_train):
            X_tr = X_train.iloc[train_idx]
            X_val = X_train.iloc[val_idx]
            y_tr = y_train.iloc[train_idx]
            y_val = y_train.iloc[val_idx]
            
            Model = LGBMRegressor(**lgbm_params).fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                          eval_metric=['rmse'],
                          early_stopping_rounds=250, 
                          categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                          #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
                          verbose=0)
            
            preds_list_base.append(Model.predict(X_test))
            preds_list_all.append(Model.predict(X_test))
            print(f'RMSE for Base model is {np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))}')
            first_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
            params = lgbm_params.copy()
            
            for i in range(1, 8):
                if i >2:    
                    
                    # reducing regularizing params if 
                    
                    params['reg_lambda'] *= 0.9
                    params['reg_alpha'] *= 0.9
                    params['num_leaves'] += 40
                    
                params['learning_rate'] = 0.003
                Model = LGBMRegressor(**params).fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                          eval_metric=['rmse'],
                          early_stopping_rounds=200, 
                          categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                          verbose=0, init_model = Model)
                
                preds_list_all.append(Model.predict(X_test))
                print(f'RMSE for Incremental trial {i} model is {np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))}')
            last_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
            print('',end='\n\n')
            print(f'Improvement of : {first_rmse - last_rmse}')
            print('-' * 100)
            preds_list_final_iteration.append(Model.predict(X_test))

RMSE for Base model is 0.8418257257072861
RMSE for Incremental trial 1 model is 0.8418030604020151
RMSE for Incremental trial 2 model is 0.8418023211901545
RMSE for Incremental trial 3 model is 0.8418027060075726
RMSE for Incremental trial 4 model is 0.8417965529244634
RMSE for Incremental trial 5 model is 0.8417945701185089
RMSE for Incremental trial 6 model is 0.8417907475685441
RMSE for Incremental trial 7 model is 0.8417908584616126


Improvement of : 3.4867245673475544e-05
----------------------------------------------------------------------------------------------------
RMSE for Base model is 0.8413736340117811
RMSE for Incremental trial 1 model is 0.8413696717637158
RMSE for Incremental trial 2 model is 0.841367552641318
RMSE for Incremental trial 3 model is 0.8413655801894794
RMSE for Incremental trial 4 model is 0.8413649795785495
RMSE for Incremental trial 5 model is 0.8413637121808708
RMSE for Incremental trial 6 model is 0.8413629580771478
RMSE for Incremental trial 7 mode

In [24]:
y_preds_base = np.array(preds_list_base).mean(axis=0)
y_preds_base

array([7.57968175, 7.75712088, 7.59559932, ..., 7.54930368, 7.49240774,
       7.29231073])

In [25]:
y_preds_all = np.array(preds_list_all).mean(axis=0)
y_preds_all

array([7.57939735, 7.75727303, 7.59217738, ..., 7.54897992, 7.49134634,
       7.29294902])

In [26]:
y_preds_final_iteration = np.array(preds_list_final_iteration).mean(axis=0)
y_preds_final_iteration

array([7.57865084, 7.75747453, 7.59047653, ..., 7.54893681, 7.49010552,
       7.29290063])

In [27]:
submission['target'] = y_preds_final_iteration

In [28]:
submission

,target
id,
0,7.578651
5,7.757475
15,7.590477
16,7.538850
17,7.282985
...,...
499987,7.523070
499990,7.233976
499991,7.548937


In [29]:
submission.to_csv("new_bl_mine.csv", index = True)
!cp new_bl_mine.csv "drive/My Drive/"